# Introduction
## Basic Tutorial II: Loop Constructs

**[AutoRA](https://pypi.org/project/autora/)** (**Au**tomated **R**esearch **A**ssistant) is an open-source framework designed to automate various stages of empirical research, including model discovery, experimental design, and data collection.

This notebook is the second of four notebooks within the basic tutorials of ``autora``. We suggest that you go through these notebooks in order as each builds upon the last. However, each notebook is self-contained and so there is no need to *run* the content of the last notebook for your current notebook. We will here provide a link to each notebook, but we will also provide a link at the end of each notebook to navigate you to the next notebook.

[AutoRA Basic Tutorial I: Components](https://autoresearch.github.io/autora/tutorials/basic/Tutorial-I-Components/) <br>
[AutoRA Basic Tutorial II: Loop Constructs](https://autoresearch.github.io/autora/tutorials/basic/Tutorial-II-Loop-Constructs/) <br>
[AutoRA Basic Tutorial III: Workflow Logic](https://autoresearch.github.io/autora/tutorials/basic/Tutorial-III-Workflow-Logic/) <br>
[AutoRA Basic Tutorial IV: Customization](https://autoresearch.github.io/autora/tutorials/basic/Tutorial-IV-Customization/) <br>

These notebooks provide a comprehensive introduction to the capabilities of ``autora``. **It demonstrates the fundamental components of ``autora``, and how they can be combined to facilitate automated (closed-loop) empirical research through synthetic experiments.**

**How to use this notebook** *You can progress through the notebook section by section or directly navigate to specific sections. If you choose the latter, it is recommended to execute all cells in the notebook initially, allowing you to easily rerun the cells in each section later without issues.*

## Tutorial Setup
This tutorial is self-contained so that you do not need to run the previous notebook to begin. However, the four notebooks are continuous so that what we define in a previous notebook should still exist within this notebook. As such, we will here re-run relevant code from past tutorials. We will not again walk you through these, but if you need a reminder what they are then go see the descriptions in previous notebooks.

In [ ]:
#### Installation ####
!pip install -q "autora[experimentalist-falsification]"
!pip install -q "autora[experimentalist-sampler-model-disagreement]"
!pip install -q "autora[theorist-bms]"

#### Import modules ####
import numpy as np
import torch
from autora.variable import DV, IV, ValueType, VariableCollection
from autora.experimentalist.pooler.random_pooler import random_pool
from autora.experimentalist.sampler.falsification import falsification_sample
from autora.experimentalist.sampler.model_disagreement import model_disagreement_sample
from autora.theorist.bms import BMSRegressor
from sklearn import linear_model

#### Set seeds ####
np.random.seed(42)
torch.manual_seed(42)

#### Define ground truth and experiment runner ####
ground_truth = lambda x: np.sin(x)
run_experiment = lambda x: ground_truth(x) + np.random.normal(0, 0.1, size=x.shape)

#### Define condition pool ####
condition_pool = np.linspace(0, 2 * np.pi, 100)
condition_pool = condition_pool.reshape((len(condition_pool), 1))

#### Define metadata ####
iv = IV(name="x", value_range=(0, 2 * np.pi), allowed_values=condition_pool)
dv = DV(name="y", type=ValueType.REAL)
metadata = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

#### Define theorists ####
theorist_lr = linear_model.LinearRegression()
theorist_bms = BMSRegressor(epochs=100)

# Loop Constructs
After defining all the components required for the empirical research process, we can create an automated workflow using basic loop constructs in Python.

The following code block demonstrates how to build such a workflow using the components introduced in the preceding notebook, such as

- ``metadata`` (object specifying variables of the experiment), <br>
- ``run_experiment`` (function for collecting data), <br>
- ``theorist_bms`` (scikit learn estimator for discoverying equations using the Bayesian Machine Scientist), <br>
- ``random_pool`` (function for generating a random pool of experimental conditions), and <br>
- ``falsification_sample`` (function for identifying novel experiment conditions using the falsification .sampler) <br>

We begin with implementing the following workflow:
1. Generate 3 seed experimental conditions using ``random_pool``
2. Generate 3 seed observations using ``run_experiment``
3. Loop through the following steps 5 times
   - Identify a model relating conditions to observations using ``theorist_bms``
   - Identify 3 new experimental conditions using ``falsification_sample``
   - Collect 3 new observations using ``run_experiment``
   - Add new conditions and observations to the dataset

We will here begin using the naming convention ``cycle`` to refer to an entire AutoRA loop where the loop encounters all AutoRA components - experiment runner, theorist, experimentalist. Within the scientific method, a cycle would then be running a single iteration of the experiment. This requires the collection of data, the modelling of that data, and the conceptualization of the next iteration of this experiment. For example, if our research concerns how much information a person acquires from a photo (dependent variable) dependent on how bright the photo is (independent variable), we may first collect data with conditions of (let's say) 10%, 50%, and 90% brightness, then model our collected data to determine the relationship between brightness and photo perception, and finally determine which other brightness conditions may help us understand the true relationship. Probing other conditions - such as a brightness of 25% and of 75% would then be the next iteration of the experiment and thus, for us, the next cycle. The following code block will iterate through five of these cycles.

## Example 1: Falsification Sampler

In [ ]:
num_cycles = 5 # number of empirical research cycles
measurements_per_cycle = 3 # number of data points to collect for each cycle

# generate an initial set of experimental conditions
conditions = random_pool(metadata.independent_variables[0].allowed_values,
                                 n=measurements_per_cycle)

# convert iterator into 2-dimensional numpy array
conditions = np.array(list(conditions)).reshape(-1, 1)

# collect initial set of observations
observations = run_experiment(conditions)

for cycle in range(num_cycles):

  # use BMS theorist to fit the model to the data
  theorist_bms.fit(conditions, observations)

  # obtain new conditions
  new_conditions = falsification_sample(
          condition_pool=condition_pool,
          model=theorist_bms,
          reference_conditions=conditions,
          reference_observations=observations,
          metadata=metadata,
          num_samples=measurements_per_cycle,
      )

  # obtain new observations
  new_observations = run_experiment(new_conditions)

  # combine old and new conditions and observations
  conditions = np.concatenate((conditions, new_conditions))
  observations = np.concatenate((observations, new_observations))

  # evaluate model of the theorist based on its ability to predict each observation from the ground truth, evaluated across the entire space of experimental conditions
  loss = np.mean(np.square(theorist_bms.predict(condition_pool) - ground_truth(condition_pool)))
  print("Loss in cycle {}: {}".format(cycle, loss))
  print("Discovered Model: " +  theorist_bms.repr())


100%|██████████| 100/100 [00:26<00:00,  3.84it/s]


Loss in cycle 0: 0.0
Discovered Model: sin(X0)


100%|██████████| 100/100 [00:28<00:00,  3.56it/s]


Loss in cycle 1: 0.0
Discovered Model: sin(X0)


100%|██████████| 100/100 [00:14<00:00,  7.10it/s]


Loss in cycle 2: 0.5526484578348648
Discovered Model: _a0_


100%|██████████| 100/100 [00:17<00:00,  5.60it/s]


Loss in cycle 3: 0.0
Discovered Model: sin(X0)


100%|██████████| 100/100 [00:16<00:00,  6.25it/s]


Loss in cycle 4: 0.0
Discovered Model: sin(X0)


## Example 2: Model Disagreement Sampler
We can easily replace components in the workflow above. For instance, we could replace  ``falsification_sample`` with the ``experimentalist_pipeline`` defined in Tutorial I.

In the following code block, we add a linear regression theorist, to fit a linear model to the data. In addition, we replace ``falsification_sample`` with  ``model_disagreement_sample`` to sample experimental conditions that differentiate most between the linear model and the model discovered by the BMS theorist.

In [ ]:
num_cycles = 5 # number of empirical research cycles
measurements_per_cycle = 3 # number of data points to collect for each cycle

# generate an initial set of experimental conditions
conditions = random_pool(metadata.independent_variables[0].allowed_values,
                                 n=measurements_per_cycle)
# convert iterator into 2-dimensional numpy array
conditions = np.array(list(conditions)).reshape(-1, 1)

# collect initial set of observations
observations = run_experiment(conditions)

for cycle in range(num_cycles):

  # use BMS theorist to fit the model to the data
  theorist_bms.fit(conditions, observations)
  theorist_lr.fit(conditions, observations)

  # obtain new conditions
  new_conditions = model_disagreement_sample(
          condition_pool,
          models = [theorist_bms, theorist_lr],
          num_samples = measurements_per_cycle
      )

  # obtain new observations
  new_observations = run_experiment(new_conditions)

  # combine old and new conditions and observations
  conditions = np.concatenate((conditions, new_conditions))
  observations = np.concatenate((observations, new_observations))

  # evaluate model of the theorist based on its ability to predict each observation from the ground truth, evaluated across the entire space of experimental conditions
  loss = np.mean(np.square(theorist_bms.predict(condition_pool) - ground_truth(condition_pool)))
  print("Loss in cycle {}: {}".format(cycle, loss))
  print("Discovered BMS Model: " +  theorist_bms.model_.__repr__())


100%|██████████| 100/100 [00:40<00:00,  2.47it/s]


Loss in cycle 0: 0.0
Discovered BMS Model: sin(X0)


100%|██████████| 100/100 [00:18<00:00,  5.53it/s]


Loss in cycle 1: 0.0
Discovered BMS Model: sin(X0)


100%|██████████| 100/100 [00:19<00:00,  5.16it/s]


Loss in cycle 2: 0.0
Discovered BMS Model: sin(X0)


100%|██████████| 100/100 [00:15<00:00,  6.37it/s]


Loss in cycle 3: 0.0
Discovered BMS Model: sin(X0)


100%|██████████| 100/100 [00:24<00:00,  4.01it/s]


Loss in cycle 4: 0.0
Discovered BMS Model: sin(X0)


# Next Notebook
While the basic loop construct is flexible, there are more convenient ways to specify a research cycle in ``autora``. The next notebook illustrates the use of these constructs.

Follow this link for the next notebook tutorial:
[AutoRA Basic Tutorial III: Workflow Logic](https://autoresearch.github.io/autora/tutorials/basic/Tutorial-III-Workflow-Logic/) <br>